# Augmenting data for neural network training

In [134]:
import numpy as np
import numpy.random as rn
import matplotlib.pyplot as plt 
import matplotlib
matplotlib.style.use('science')

from astropy.io import fits
from astropy.table import Table, vstack
import pandas as pd
from matplotlib import colors

import warnings

Here we have a sample table of observed stars, with known ages and Av. We want to vastly increase the size of this sample.

In [199]:
final2 = fits.open('Data/Train/final2.fits')
trData = Table(final2[1].data)
np.random.seed(0); trData = trData[np.random.permutation(len(trData))]#.to_pandas()
#Table is sorted by age - that's no good! randomly sample, but try to keep the same with a given seed

#I don't want to preserve the source_id and things like that, create new table and set to 0
trData

SOURCE_ID,LABELS,PROB,RA,DEC,PARALLAX,VLSRRA,VLSRDEC,BP_RP,ABSG,L,L1,B,AGE,AV,PHOT_G_MEAN_FLUX,PHOT_G_MEAN_FLUX_ERROR,PHOT_G_MEAN_MAG,PHOT_BP_MEAN_FLUX,PHOT_BP_MEAN_FLUX_ERROR,PHOT_BP_MEAN_MAG,PHOT_RP_MEAN_FLUX,PHOT_RP_MEAN_FLUX_ERROR,PHOT_RP_MEAN_MAG,PARALLAX_ERROR,VLSRL,VLSRB,SLABEL,RADIAL_VELOCITY,RADIAL_VELOCITY_ERROR,VLSRV,J_M,J_MSIGCOM,H_M,H_MSIGCOM,KS_M,KS_MSIGCOM,W1MPRO,W1MPRO_ERROR,W2MPRO,W2MPRO_ERROR,W3MPRO,W3MPRO_ERROR,W4MPRO,W4MPRO_ERROR,NAME,PLOTNAME,ID,teff_val,a_g_val,a_g_percentile_lower,a_g_percentile_upper,e_bp_min_rp_val,e_bp_min_rp_percentile_lower,e_bp_min_rp_percentile_upper,radius_val,radius_percentile_lower,radius_percentile_upper,lum_val,lum_percentile_lower,lum_percentile_upper
int64,int32,float32,float64,float64,float32,float32,float32,float32,float32,float64,float64,float64,float32,float32,float64,float64,float32,float64,float64,float32,float64,float64,float32,float64,float32,float32,int32,float64,float64,float64,float32,float32,float32,float32,float32,float32,float64,float64,float64,float64,float64,float64,float64,float64,str17,str23,int32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32
4065243621545087616,1645,0.90739787,275.19848734244766,-24.769855073662466,2.8322957,-5.6046295,-3.1828508,1.8355827,7.961492,7.491534513103659,7.491534513103659,-4.779078946425696,8.216881,0.7344418,9886.141715903206,7.372325319608128,15.700799,3211.9074049358283,22.569017540708376,16.58448,10120.667634650876,44.095890947755045,14.748898,0.052369733531658136,-5.443862,3.4506204,98,nan,nan,nan,13.512,0.028,12.804,0.036,12.579,0.04,nan,nan,nan,nan,nan,nan,nan,nan,,Theia 459,459,3984.0,0.3422,0.1119,0.5328,0.166,0.047,0.233,0.6009419,0.5514337,0.62678593,0.08196661,0.07865764,0.085275576
5847411984240916864,1825,0.39078614,213.831286068137,-68.62605443532621,3.4100044,-3.9035578,-6.711186,1.8574114,7.9138746,310.4834881800583,-49.516511819941684,-6.993047433063589,7.4674296,0.26631397,14972.90588873749,26.958641900611912,15.2501,4828.125301641481,37.22707251013589,16.141941,15522.304381512507,88.38481589723263,14.28453,0.03210695159984592,-5.813508,-5.1459603,19,nan,nan,nan,12.87,0.058,12.176,0.07,12.012,0.052,nan,nan,nan,nan,nan,nan,nan,nan,BH_164,BH_164 (75),75,3928.5,0.691,0.4523,0.8511,0.344,0.242,0.4445,0.64371896,0.4843941,0.68980485,0.088918954,0.086913474,0.090924434
5237123455370965632,82,0.32023832,174.3660389346444,-66.42129868174743,6.6427574,0.4993412,10.444072,nan,nan,295.61878136916846,-64.38121863083154,-4.605541587423504,7.549037,0.24329777,1914.36372964352,2.2289944928656635,17.483305,181.05237713404847,10.458707445285377,19.706877,2977.217656658268,11.325157445426717,16.077393,0.1081613601252883,-2.4663587,10.160957,21,nan,nan,nan,13.876,0.024,13.228,0.022,12.973,0.038,12.714,0.025,12.531,0.024,12.517,0.283,9.627,nan,IC_2602,IC_2602 (92),92,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
5514367614389714688,301,0.91152424,122.65463557601619,-49.28297507683276,2.5938716,-2.942635,1.434512,2.0400276,7.9762545,264.5642365331618,-95.43576346683818,-8.568789745932017,7.496,0.201,8179.773487424819,8.06728051615593,15.906512,2341.117095946072,18.848881670746447,16.92783,8905.270707515423,23.281121483824986,14.887802,0.04109638538675536,-2.790596,-1.7115784,24,nan,nan,nan,13.497,0.026,12.812,0.027,12.614,0.027,12.564,0.023,12.539,0.022,12.531,0.414,9.04,nan,NGC_2547,NGC_2547 (74),74,4053.615,1.4412,1.134,1.861,0.7303,0.5637,0.9154,0.56450045,0.3842329,0.6830843,0.07751634,0.07472826,0.08030442
4313298152031608192,129,0.91024566,288.4170099143132,12.904721772298762,3.149274,-6.8295403,-18.235126,2.6103306,9.304495,46.92728458743346,46.92728458743346,1.0212508783551109,8.952694,0.4142455,3547.883581502598,3.0151151284410647,16.813442,704.2649555714572,8.315872855766193,18.232048,4529.820771535895,13.51562351170223,15.621717,0.0846269933550978,-19.31118,-2.4981537,-1,nan,nan,nan,13.878,0.031,13.211,0.031,12.937,0.034,nan,nan,nan,nan,nan,nan,nan,nan,,T

Turning fluxes into magnitudes in Gaia: http://dc.zah.uni-heidelberg.de/tableinfo/gaia.dr2epochflux (useful vega mags)

In [232]:
def NewTablePhotErr(table):
    tab = table[:]#Make sure it's obvious these points aren't real!
    tab['SOURCE_ID']=0
    
    vegaMag = {'PHOT_G_MEAN_FLUX':25.6884, 'PHOT_BP_MEAN_FLUX':25.3514, 'PHOT_RP_MEAN_FLUX':24.7619} #magnitudes for these specific fluxes
    fluxCols = [('PHOT_G_MEAN_FLUX', 'PHOT_G_MEAN_FLUX_ERROR','PHOT_G_MEAN_MAG'), ('PHOT_BP_MEAN_FLUX', 'PHOT_BP_MEAN_FLUX_ERROR','PHOT_BP_MEAN_MAG'),
                ('PHOT_RP_MEAN_FLUX', 'PHOT_RP_MEAN_FLUX_ERROR','PHOT_RP_MEAN_MAG')]
    magCols = [('J_M', 'J_MSIGCOM'), ('H_M', 'H_MSIGCOM'), ('KS_M', 'KS_MSIGCOM'), ('W1MPRO', 'W1MPRO_ERROR'), ('W2MPRO', 'W2MPRO_ERROR')]
    
    np.warnings.filterwarnings('ignore')
    for flux, error, mag in fluxCols:
        F = np.array(tab[flux] + rn.normal(scale = tab[error]))
        tab.replace_column(flux, F)
        M = np.array(-2.5 * np.log10(tab[flux]) + vegaMag[flux])
        tab.replace_column(mag, M)

    for mag, error in magCols:
        M = np.array(tab[mag] + rn.normal(scale = tab[error]))
        tab.replace_column(mag, M)
    return tab

In [223]:
def NewTablePrlxErr(table):
    tab = table[:]#Make sure it's obvious these points aren't real!
    tab['SOURCE_ID']=0
    
    P = np.array(tab['PARALLAX'] + rn.normal(scale = tab['PARALLAX_ERROR']))
    tab.replace_column('PARALLAX', p)
    return tab

Extinction changes magnitudes by a predictable amount and has no effect on other parameters, so we can add extinction easily

In [195]:
def NewTableAV(table, avMax = 10):
    tab = table[:]
    tab['SOURCE_ID']=0
    
    ext = {'J_M': 0.288,
     'H_M': 0.178,
     'KS_M': 0.117,
     'W1MPRO': 0.078,
     'W2MPRO': 0.044,
     'PHOT_G_MEAN_MAG': 0.859,
     'PHOT_BP_MEAN_MAG': 1.068,
     'PHOT_RP_MEAN_MAG': 0.652}
    
    moreAv = rn.rand(len(tab))*avMax
    for label in ext.keys():
        extinguished = tab[label]-moreAv * ext[label]
        tab.replace_column(label, extinguished)
        
    newAv = tab['AV']-moreAv
    tab.replace_column('AV', moreAv)
    return tab

Changing parallax (distance) involves a shift in the distance modulus - magnitudes
$$m1 = 5\log(\frac{\pi_2}{\pi_1}) + M2$$

In [208]:
def NewTableParallax(table, min=.5, max=10):
    tab = table[:]
        tab['SOURCE_ID']=0
    
    parallax = tab['PARALLAX']
    newParallax = rn.rand(len(tab))*(max-min)+min
    
    mags = ['J_M','H_M','KS_M','W1MPRO','W2MPRO','PHOT_G_MEAN_MAG','PHOT_BP_MEAN_MAG','PHOT_RP_MEAN_MAG']
    for label in mags:
        M = tab[label] + 5 * np.log10(parallax/newParallax)
        tab.replace_column(label, M)
        
    tab.replace_column('PARALLAX', newParallax)
    return tab

Now we can increase our sample size drastically

In [237]:
data = trData[:]
for i in range(4):
    data = vstack([data, NewTablePhotErr(trData)])
for i in range(2):
    data = vstack([data, NewTableAV(trData)])
    data = vstack([data, NewTableParallax(trData)])

In [238]:
data

SOURCE_ID,LABELS,PROB,RA,DEC,PARALLAX,VLSRRA,VLSRDEC,BP_RP,ABSG,L,L1,B,AGE,AV,PHOT_G_MEAN_FLUX,PHOT_G_MEAN_FLUX_ERROR,PHOT_G_MEAN_MAG,PHOT_BP_MEAN_FLUX,PHOT_BP_MEAN_FLUX_ERROR,PHOT_BP_MEAN_MAG,PHOT_RP_MEAN_FLUX,PHOT_RP_MEAN_FLUX_ERROR,PHOT_RP_MEAN_MAG,PARALLAX_ERROR,VLSRL,VLSRB,SLABEL,RADIAL_VELOCITY,RADIAL_VELOCITY_ERROR,VLSRV,J_M,J_MSIGCOM,H_M,H_MSIGCOM,KS_M,KS_MSIGCOM,W1MPRO,W1MPRO_ERROR,W2MPRO,W2MPRO_ERROR,W3MPRO,W3MPRO_ERROR,W4MPRO,W4MPRO_ERROR,NAME,PLOTNAME,ID,teff_val,a_g_val,a_g_percentile_lower,a_g_percentile_upper,e_bp_min_rp_val,e_bp_min_rp_percentile_lower,e_bp_min_rp_percentile_upper,radius_val,radius_percentile_lower,radius_percentile_upper,lum_val,lum_percentile_lower,lum_percentile_upper,source_id
int64,int32,float32,float64,float64,float64,float32,float32,float32,float32,float64,float64,float64,float32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float32,float32,int32,float64,float64,float64,float64,float32,float64,float32,float64,float32,float64,float64,float64,float64,float64,float64,float64,float64,str17,str23,int32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int32
0,1645,0.90739787,275.19848734244766,-24.769855073662466,2.8322956562042236,-5.6046295,-3.1828508,1.8355827,7.961492,7.491534513103659,7.491534513103659,-4.779078946425696,8.216881,0.7344418168067932,9886.141715903206,7.372325319608128,15.700798988342285,3211.9074049358283,22.569017540708376,16.58448028564453,10120.667634650876,44.095890947755045,14.748897552490234,0.052369733531658136,-5.443862,3.4506204,98,nan,nan,nan,13.51200008392334,0.028,12.803999900817871,0.036,12.579000473022461,0.04,nan,nan,nan,nan,nan,nan,nan,nan,,Theia 459,459,3984.0,0.3422,0.1119,0.5328,0.166,0.047,0.233,0.6009419,0.5514337,0.62678593,0.08196661,0.07865764,0.085275576,--
0,1825,0.39078614,213.831286068137,-68.62605443532621,3.4100043773651123,-3.9035578,-6.711186,1.8574114,7.9138746,310.4834881800583,-49.516511819941684,-6.993047433063589,7.4674296,0.26631397008895874,14972.90588873749,26.958641900611912,15.250100135803223,4828.125301641481,37.22707251013589,16.14194107055664,15522.304381512507,88.38481589723263,14.284529685974121,0.03210695159984592,-5.813508,-5.1459603,19,nan,nan,nan,12.869999885559082,0.058,12.175999641418457,0.07,12.01200008392334,0.052,nan,nan,nan,nan,nan,nan,nan,nan,BH_164,BH_164 (75),75,3928.5,0.691,0.4523,0.8511,0.344,0.242,0.4445,0.64371896,0.4843941,0.68980485,0.088918954,0.086913474,0.090924434,--
0,82,0.32023832,174.3660389346444,-66.42129868174743,6.642757415771484,0.4993412,10.444072,nan,nan,295.61878136916846,-64.38121863083154,-4.605541587423504,7.549037,0.2432977706193924,1914.36372964352,2.2289944928656635,17.483304977416992,181.05237713404847,10.458707445285377,19.706876754760742,2977.217656658268,11.325157445426717,16.077392578125,0.1081613601252883,-2.4663587,10.160957,21,nan,nan,nan,13.87600040435791,0.024,13.227999687194824,0.022,12.972999572753906,0.038,12.714,0.025,12.531,0.024,12.517,0.283,9.627,nan,IC_2602,IC_2602 (92),92,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,--
0,301,0.91152424,122.65463557601619,-49.28297507683276,2.593871593475342,-2.942635,1.434512,2.0400276,7.9762545,264.5642365331618,-95.43576346683818,-8.568789745932017,7.496,0.20100000500679016,8179.773487424819,8.06728051615593,15.906512260437012,2341.117095946072,18.848881670746447,16.92782974243164,8905.270707515423,23.281121483824986,14.887802124023438,0.04109638538675536,-2.790596,-1.7115784,24,nan,nan,nan,13.496999740600586,0.026,12.812000274658203,0.027,12.61400032043457,0.027,12.564,0.023,12.539,0.022,12.531,0.414,9.04,nan,NGC_2547,NGC_2547 (74),74,4053.615,1.4412,1.134,1.861,0.7303,0.5637,0.9154,0.56450045,0.3842329,0.6830843,0.07751634,0.07472826,0.08030442,--
0,129,0.91024566,288.4170099143132,12.904721772298762,3.1492741107940674,-6.8295403,-18.235126,2.6103306,9.304495,46.92728458743346,46.92728458743346,1.0212508783551109,8.952694,0.41424

In [239]:
2595330/288370

9.0